In [ ]:
import nibabel as nib
import numpy as np
import os
from skimage.measure import compare_ssim as cssim
from skimage.measure import compare_psnr as cpsnr

In [ ]:
# Data paths for ground truth, degraded, and super-resolved test batch.
input_path_gt = 'D:/Data/IXI-T1-GT/'
input_path_degr = 'D:/Data/IXI-T1-DEGR/'
input_path_sr = 'D:/Data/IXI-T1-SR/'

# Lists for PSNR and SSIM comparisons between ground truth - degraded and 
# ground truth - super-resolved volumetric images.
psnr_gt_degr = []
ssim_gt_degr = []
psnr_gt_sr = []
ssim_gt_sr = []

for root, dirs, filenames in os.walk(input_path_degr):
    for f in filenames:
        print(f)

        # Loading of degraded volumetric image.
        fullpath_degr = os.path.join(input_path_degr, f)
        img_degr = nib.load(fullpath_degr)
        data_degr = img_degr.get_data().astype(np.float64)

        # Loading of ground truth volumetric image.
        fullpath_gt = os.path.join(input_path_gt,f)
        img_gt = nib.load(fullpath_gt)
        data_gt = img_gt.get_data().astype(np.float64)        

        # Computation of PSNR and SSIM between ground truth and degraded volumetric images.
        # Settings "gaussian_weights", "sigma", and "use_sample_covariance" are set according to [1].
        # [1] http://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.compare_ssim
        psnr_gt_degr.append(cpsnr(data_gt, data_degr, data_range = np.amax(data_gt) - np.amin(data_gt)))
        ssim_gt_degr.append(cssim(data_gt, data_degr, data_range = np.amax(data_gt) - np.amin(data_gt), gaussian_weights = True, sigma = 1.5, use_sample_covariance = False))   

        # Loading of super-resolved volumetric image.
        f2 = f.replace('-', '')[:-7]+'_niftynet_out.nii.gz'        
        fullpath_sr = os.path.join(input_path_sr, f2)
        img_sr = nib.load(fullpath_sr)
        data_sr = img_sr.get_data().astype(np.float64)[:,:,:,0,0]      

        # Computation of PSNR and SSIM between ground truth and super-resolved volumetric images.
        psnr_gt_sr.append(cpsnr(data_gt, data_sr, data_range = np.amax(data_gt) - np.amin(data_gt)))
        ssim_gt_sr.append(cssim(data_gt, data_sr, data_range = np.amax(data_gt) - np.amin(data_gt), gaussian_weights = True, sigma = 1.5, use_sample_covariance = False))   

# Printing of average PSNR and SSIM for the respective comparisons.
print(np.mean(psnr_gt_degr),np.mean(ssim_gt_degr))
print(np.mean(psnr_gt_sr),np.mean(ssim_gt_sr))